In [19]:
import os
import pandas as pd

def process_csv_files(input_folder, output_file):
    combined_df = pd.DataFrame()
    
    # Loop through each CSV file in the input folder
    for filename in os.listdir(input_folder):
        if filename.endswith('.csv'):
            filepath = os.path.join(input_folder, filename)
            df = pd.read_csv(filepath, header=None)
            
            # Step 1: Remove the first column
            df = df.iloc[:, 1:]
            
            # Step 2: Merge the first 4 columns with a delimiter "||"
            df['Merged'] = df.iloc[:, 0].astype(str) + '~' + df.iloc[:, 1].astype(str) + '~' + df.iloc[:, 2].astype(str) + '~' + df.iloc[:, 3].astype(str)
            
            # Step 3: Bring the merged column to the first position
            cols = ['Merged'] + [col for col in df.columns if col != 'Merged']
            df = df[cols]
            
            # Combine the first two rows to create the new header with "||" separator
            new_header = [f"{df.iloc[0, col]}~{df.iloc[1, col]}" if pd.notna(df.iloc[0, col]) or pd.notna(df.iloc[1, col]) else '' 
                          for col in range(len(df.columns))]
            
            # Create a new DataFrame with the combined header and the remaining rows
            df_combined = pd.DataFrame(df.values[2:], columns=new_header)
            
            # Step 4: Remove the dimension columns else than merged
            df = df_combined
            df.drop(df.columns[1:5], axis=1, inplace=True)
            
            # Extract the first column as attributes
            attributes = df.iloc[:, 0].values
            
            # Extract the data to be repeated
            data = df.iloc[:, 1:].values
            
            # Create a list to hold the transformed data
            transformed_data = []
            
            # Iterate over each attribute
            for i, attribute in enumerate(attributes):
                # Iterate over each column of the remaining data
                for j in range(data.shape[1]):
                    transformed_data.append([attribute, df.columns[j + 1], data[i, j]])
            
            # Create a new DataFrame from the transformed data
            df = pd.DataFrame(transformed_data, columns=['Movie/Series Title||Content Title||Season Number||Episode Number', 'Metric', 'Value'])
            # Split the Metric column into two separate columns: Date and Attribute using "||" as delimiter
            df[['Date', 'Attribute']] = df['Metric'].str.split('~', expand=True)
            # Drop the original Metric column
            df = df.drop(columns=['Metric'])
            
            # Pivot the DataFrame
            df_pivot = df.pivot_table(index=['Movie/Series Title||Content Title||Season Number||Episode Number', 'Date'], 
                                       columns='Attribute', 
                                       values='Value',
                                       aggfunc='first').reset_index()
            # Flatten the MultiIndex columns
            df_pivot.columns.name = None
            df = df_pivot
            
            # Split the first column into separate columns
            df_columns = df.iloc[:, 0].str.split('~', expand=True)
            # Assign meaningful names to the new columns
            df_columns.columns = ['Movie/Series Title', 'Content Title', 'Season Number', 'Episode Number']
            # Drop the original first column and concatenate the new columns
            df = pd.concat([df_columns, df.drop(columns=df.columns[0])], axis=1)
            # Multiply Streaming Hours Sum by 60 to convert to Minutes
            df['Streaming Hours Sum'] = pd.to_numeric(df['Streaming Hours Sum'], errors='coerce') * 60
            # Rename the columns
            df = df.rename(columns={
                'Stream Start Event Count': 'Views',
                'Streaming Hours Sum': 'Minutes Viewed'
            })
            # Convert the Date column to MM-DD-YYYY format
            df['Date'] = pd.to_datetime(df['Date']).dt.strftime('%m-%d-%Y')
            
            # Append the processed DataFrame to the combined DataFrame
            combined_df = pd.concat([combined_df, df], ignore_index=True)
    
    # Save the combined DataFrame to an Excel file
    combined_df.to_csv(output_file, index=False)

# Set the input and output folder paths
input_folder = 'C:/Users/smondal/OneDrive - Cineverse/LINEAR DATA CALCULATION METRIC/ROKU FILE CREATION/Input'
output_folder = 'C:/Users/smondal/OneDrive - Cineverse/LINEAR DATA CALCULATION METRIC/ROKU FILE CREATION/Output'
output_file = os.path.join(output_folder, 'output.csv')
process_csv_files(input_folder, output_file)